![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [3]:
from keras.utils import serialize_keras_object
from keras import utils
from keras.models import Sequential
from keras.layers import Dense, Flatten
import json
from datetime import datetime
import pandas as pd

In [4]:
start = datetime(2020, 1, 1)
end = datetime(2022, 1, 1)

In [5]:
qb = QuantBook()
symbol = qb.AddCrypto("BTCUSD", Resolution.Daily).Symbol
history = qb.History(symbol, start, end).loc[symbol]
history.head()

In [6]:
daily_pct_change = history[["open", "high", "low", "close", "volume"]].pct_change().dropna()
df = daily_pct_change
df.head()

In [7]:
indexes = df[((df.volume == float("inf")))].index
for i in indexes:
    df.at[i, "volume"] = max(df.volume.drop(indexes))

In [8]:
n_steps = 30
features = []
labels = []
for i in range(len(df)-n_steps):
    input_data = df.iloc[i:i+n_steps].values
    features.append(input_data)
    if df['close'].iloc[i+n_steps] >= 0:
        # UP
        label = 1
    else:
        # DOWN
        label = 0
    labels.append(label)

In [9]:
features = np.array(features)
labels = np.array(labels)

In [10]:
train_length = int(len(features) * 0.7)
X_train = features[:train_length]
X_test = features[train_length:]
y_train = labels[:train_length]
y_test = labels[train_length:]

In [11]:
# number of up vs down days in training data should be relatively balanced
sum(y_train)/len(y_train)

In [12]:
# use second part of data for training instead
test_length = int(len(features) * 0.3)
X_train = features[test_length:]
X_test = features[:test_length]
y_train = labels[test_length:]
y_test = labels[:test_length]

In [13]:
sum(y_train)/len(y_train)

In [14]:
model = Sequential([Dense(30, input_shape=X_train[0].shape, activation='relu'),
                    Dense(20, activation='relu'),
                    Flatten(),
                    Dense(1, activation='sigmoid')])

In [15]:
model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy', 'mse'])

In [16]:
model.fit(X_train, y_train, epochs=5)

In [17]:
y_hat = model.predict(X_test)

In [18]:
results = pd.DataFrame({'y': y_test.flatten(), 'y_hat': y_hat.flatten()})

In [19]:
results.plot(title='Model Performance: predicted vs actual %change in closing price', figsize=(17, 7))

In [20]:
pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))

pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))

### Save Model

In [21]:
model_str = json.dumps(serialize_keras_object(model))

In [22]:
model_key = 'bitcoin_price_predictor'

In [23]:
qb.ObjectStore.Save(model_key, model_str)

### Load Model

In [24]:
if qb.ObjectStore.ContainsKey(model_key):
    model_str = qb.ObjectStore.Read(model_key)
    config = json.loads(model_str)['config']
    model = Sequential.from_config(config)

In [25]:
testDate = datetime.now()

In [26]:
df = qb.History(symbol, testDate - timedelta(40), testDate).loc[symbol]
df_change = df[["open", "high", "low", "close", "volume"]].pct_change().dropna()
model_input = []
for index, row in df_change.tail(30).iterrows():
    model_input.append(np.array(row))
model_input = np.array([model_input])

In [27]:
if round(model.predict(model_input)[0][0]) == 0:
    print("Bearish")
else:
    print("Bullish")